In [1]:
!pip install sagemaker==1.72.0

     |████████████████████████████████| 297 kB 18.5 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=386358 sha256=09015e8c6c5bd466f058e32c6f45685fef2403bd817761e083b53fed81d75a50
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 1.0.1
    Uninstalling smdebug-rulesconfig-1.0.1:
      Successfully uninstalled smdebug-rulesconfig-1.0.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.45.0
    Uninstalling sagemaker-2.45.0:
      Successfully uninstalled sagemaker-2.45.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [8]:
import sagemaker
import os
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

session = sagemaker.Session()
role = get_execution_role()
bucket = session.default_bucket()

In [5]:
prefix = 'starbucks-offer'
data_dir = '/home/ec2-user/SageMaker/ML_project_starbucksOffer/Data'
 = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [16]:
from sagemaker.sklearn.estimator import SKLearn

output_path = 's3://{}/{}'.format(bucket, prefix)
trainScript = os.path.join(data_dir, 'train_decisionTree.py')
estimator = SKLearn(
    #container,
    entry_point='train_decisionTree.py',
    source_dir=data_dir,
    framework_version = '0.20.0',
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    #sagemaker_session=sagemaker_session,
    #output_path=output_path
)




This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [17]:
estimator.fit({'train': train_location})



's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2021-08-09 17:35:59 Starting - Starting the training job...
2021-08-09 17:36:01 Starting - Launching requested ML instances...
2021-08-09 17:36:55 Starting - Preparing the instances for training.........
2021-08-09 17:38:10 Downloading - Downloading input data...
2021-08-09 17:38:50 Training - Downloading the training image..2021-08-09 17:39:07,078 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-08-09 17:39:07,080 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 17:39:07,090 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-08-09 17:39:07,837 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 17:39:07,849 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 17:39:07,861 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-08-09 17:39:07,871 sagemaker-

In [18]:
%%time

predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!CPU times: user 271 ms, sys: 5.89 ms, total: 276 ms
Wall time: 7min 32s


In [34]:
def get_recall_precision_matrix(y_test, y_test_pred):
    """
        Compare test label and predicted label and
        reutrn recall / precision score.
    """
    tp = fp = tn = fn = 0
    for i in range(len(y_test_pred)):
        if y_test.values[i] == y_test_pred[i] == 1:
            tp += 1
        elif y_test.values[i] == y_test_pred[i] == 0:
            tn += 1
        elif y_test.values[i] == 1 and y_test_pred[i] == 0:
            fp += 1
        else:
            fn += 1
        
    recall = tp/(tp+fp)
    precision = tp/(tp+fn)

    return {"recall":recall,"precision":precision}

In [22]:
import pandas as pd
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [25]:
test_y_preds = predictor.predict(test_x)

In [35]:
#Get recall score
# In this case we will focus on high recall score as we want to maximize the possibilty
# of capturing customers who would like to accept the offer 
get_recall_precision_matrix(test_y,test_y_preds)


{'recall': 0.8903532461920709, 'precision': 0.7782079123784346}

In [37]:
import boto3
deployment_name = 'sagemaker-scikit-learn-2021-08-09-17-35-54-935'
client = boto3.client('sagemaker')
response = client.describe_endpoint_config(EndpointConfigName=deployment_name)
model_name = response['ProductionVariants'][0]['ModelName']

In [40]:
# Clean up resources
client.delete_model(ModelName=model_name)    
client.delete_endpoint(EndpointName=deployment_name)
client.delete_endpoint_config(EndpointConfigName=deployment_name)

{'ResponseMetadata': {'RequestId': '4346879b-12a3-446f-a06a-83785a823001',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4346879b-12a3-446f-a06a-83785a823001',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 09 Aug 2021 18:22:35 GMT'},
  'RetryAttempts': 0}}